# Checkpoint 8: RAM Reading and N64 Memory Mapping

In this notebook, we'll learn how to read game state directly from the N64's RAM. This is a powerful technique that provides precise information about game state without needing computer vision.

**Learning Objectives:**
1. Understand why RAM reading is valuable for RL
2. Learn N64 memory architecture basics
3. Discover Mario Kart 64 RAM addresses
4. Implement a RAM reader class
5. Visualize game state from memory data

---
## 1. Setup and Installations

In [ ]:
!pip install -q numpy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import struct
from typing import Optional, Dict, Any

print(f"numpy version: {np.__version__}")

---
## 2. Theory: Why Read RAM vs Screen Pixels?

### The Traditional Approach: Screen Pixels

Most RL agents for video games use **screen pixels** as observations:

```
Screen (320x240 RGB) -> CNN -> State Representation -> Policy
```

**Advantages of pixels:**
- Works for any game without reverse engineering
- Closer to how humans play
- No game-specific knowledge needed

**Disadvantages of pixels:**
- High dimensional input (320 x 240 x 3 = 230,400 values)
- Requires expensive CNN processing
- Imprecise information (speed shown as a visual bar, not exact number)
- Noise from visual effects, particles, etc.

---

### The RAM Reading Approach

Instead, we can read **game state directly from memory**:

```
RAM Addresses -> [speed, position_x, position_y, lap, checkpoint] -> Policy
```

**Advantages of RAM reading:**
- **Speed**: Reading a few bytes is much faster than processing 230K pixels
- **Precision**: Exact values (speed = 45.7) vs visual estimation
- **Low-dimensional**: Only need ~10-20 values instead of 230K
- **No vision needed**: No CNN required, simpler models work
- **No noise**: Visual effects don't affect readings

**Disadvantages:**
- Requires reverse engineering game memory
- Game-specific (addresses vary by game/version)
- Miss some information that's only visual

---

### Performance Comparison

| Aspect | Screen Pixels | RAM Reading |
|--------|--------------|-------------|
| Observation size | 230,400 floats | 10-20 floats |
| Processing time | ~5-10ms (CNN) | ~0.01ms |
| Precision | Visual estimate | Exact value |
| Model complexity | Deep CNN required | Simple MLP works |
| Training samples | Millions needed | Thousands may suffice |

For Mario Kart 64, RAM reading gives us a **significant advantage** because:
1. We need precise speed and position for good reward shaping
2. Lap counting and checkpoint tracking is trivial from RAM
3. We can detect collisions, items, and other states instantly

---
## 3. Theory: N64 Memory Layout Basics

### N64 Hardware Overview

The Nintendo 64 has:
- **CPU**: 64-bit NEC VR4300 @ 93.75 MHz
- **RAM**: 4 MB RDRAM (expandable to 8 MB with Expansion Pak)
- **Architecture**: Big-endian byte order

### Memory Map

```
Address Range        | Description
---------------------|-----------------------
0x00000000-0x03FFFFFF | RDRAM (physical memory)
0x04000000-0x04000FFF | SP DMEM (RSP data memory)
0x04001000-0x04001FFF | SP IMEM (RSP instruction memory)
0x04040000-0x040FFFFF | SP Registers
0x04100000-0x041FFFFF | DP Command Registers
0x04300000-0x043FFFFF | MI Registers
0x04400000-0x044FFFFF | VI Registers (video)
0x04500000-0x045FFFFF | AI Registers (audio)
0x04600000-0x046FFFFF | PI Registers (peripheral)
0x04700000-0x047FFFFF | RI Registers (RDRAM)
0x04800000-0x048FFFFF | SI Registers (serial)
0x10000000-0x1FBFFFFF | Cartridge ROM
```

### Virtual Memory

Games use virtual addresses that map to physical RAM:
- **KSEG0** (0x80000000-0x9FFFFFFF): Cached, maps to 0x00000000
- **KSEG1** (0xA0000000-0xBFFFFFFF): Uncached, maps to 0x00000000

Most game variables are in **KSEG0** space, so addresses typically start with `0x80`.

### Emulator Memory Access

Mupen64Plus exposes memory through its API:
```python
# The emulator maps N64 RAM starting at some base address
# To read address 0x800F6BBC, we calculate:
# offset = 0x800F6BBC - 0x80000000 = 0x000F6BBC
# Then read from emulator_base + offset
```

---
## 4. Mario Kart 64 RAM Addresses

Through community reverse engineering efforts, many important RAM addresses have been documented.

### Key Addresses (US NTSC Version)

| Address | Type | Description | Notes |
|---------|------|-------------|-------|
| `0x800F6BBC` | float | Player 1 Speed | Current velocity magnitude |
| `0x800F6898` | float | Player 1 X Position | 3D world coordinate |
| `0x800F689C` | float | Player 1 Y Position | Height/vertical |
| `0x800F68A0` | float | Player 1 Z Position | 3D world coordinate |
| `0x800F68B0` | float | Player 1 X Rotation | Facing direction X |
| `0x800F68B4` | float | Player 1 Y Rotation | Facing direction Y |
| `0x800F68B8` | float | Player 1 Z Rotation | Facing direction Z |
| `0x800DC5A0` | byte | Current Lap | 0-indexed (0=lap 1, 1=lap 2, 2=lap 3) |
| `0x800DC5AC` | byte | Race Position | 1-8 (1st through 8th place) |
| `0x800F6860` | short | Checkpoint ID | Track progress indicator |
| `0x800DC598` | byte | Race Finished Flag | 0=racing, 1=finished |
| `0x800F6BC0` | float | Max Speed | Maximum achievable speed |
| `0x800F68DC` | byte | Item Held | Current item in inventory |
| `0x800DCDB0` | byte | Coins | Number of coins collected |

### Player Offsets

For multiplayer, each player's data is offset:
- Player 1: Base address
- Player 2: Base + 0x100
- Player 3: Base + 0x200
- Player 4: Base + 0x300

### Address Verification

These addresses are for the **US NTSC version**. Other versions (PAL, Japanese) will have different addresses. Always verify by:
1. Checking game header in ROM
2. Testing known value changes
3. Comparing with documented addresses for your version

In [ ]:
# Define address constants for Mario Kart 64 (US NTSC)
class MarioKart64Addresses:
    """
    RAM addresses for Mario Kart 64 (US NTSC version).
    
    All addresses are in N64 virtual memory space (0x80XXXXXX).
    To get physical offset, subtract 0x80000000.
    """
    
    # Base address for N64 RAM in emulator
    KSEG0_BASE = 0x80000000
    
    # Player 1 state
    P1_SPEED = 0x800F6BBC        # float: Current speed
    P1_POS_X = 0x800F6898        # float: X world position
    P1_POS_Y = 0x800F689C        # float: Y world position (height)
    P1_POS_Z = 0x800F68A0        # float: Z world position
    P1_ROT_X = 0x800F68B0        # float: X rotation
    P1_ROT_Y = 0x800F68B4        # float: Y rotation
    P1_ROT_Z = 0x800F68B8        # float: Z rotation
    P1_MAX_SPEED = 0x800F6BC0    # float: Maximum speed
    P1_ITEM = 0x800F68DC         # byte: Current item
    
    # Race state
    CURRENT_LAP = 0x800DC5A0     # byte: Current lap (0-indexed)
    RACE_POSITION = 0x800DC5AC   # byte: Position in race (1-8)
    CHECKPOINT = 0x800F6860      # short: Current checkpoint ID
    RACE_FINISHED = 0x800DC598   # byte: Race finished flag
    COINS = 0x800DCDB0           # byte: Coin count
    
    # Timer
    TOTAL_TIME = 0x800DC540      # Total race time
    LAP_TIME = 0x800DC544        # Current lap time
    
    # Player offset for multiplayer
    PLAYER_OFFSET = 0x100
    
    @classmethod
    def physical_offset(cls, virtual_addr: int) -> int:
        """Convert N64 virtual address to physical RAM offset."""
        return virtual_addr - cls.KSEG0_BASE

# Print address table
print("Mario Kart 64 RAM Addresses (US NTSC)")
print("=" * 50)
addrs = MarioKart64Addresses()
for name in dir(addrs):
    if not name.startswith('_') and name.isupper() and name != 'KSEG0_BASE' and name != 'PLAYER_OFFSET':
        value = getattr(addrs, name)
        if isinstance(value, int):
            offset = addrs.physical_offset(value)
            print(f"{name:20s}: 0x{value:08X} (offset: 0x{offset:06X})")

---
## 5. Theory: Big-Endian Byte Order

### What is Endianness?

**Endianness** determines how multi-byte values are stored in memory:

- **Big-endian**: Most significant byte first (used by N64, network protocols)
- **Little-endian**: Least significant byte first (used by x86, ARM)

### Example: Storing 0x12345678

```
Address:      0x00   0x01   0x02   0x03
             +------+------+------+------+
Big-endian:  | 0x12 | 0x34 | 0x56 | 0x78 |  (MSB first)
             +------+------+------+------+
Little-endian: | 0x78 | 0x56 | 0x34 | 0x12 |  (LSB first)
             +------+------+------+------+
```

### N64 is Big-Endian

The N64 CPU (MIPS R4300i) uses **big-endian** byte order. When we read multi-byte values from N64 RAM, we need to interpret them as big-endian.

### Reading Floats

IEEE 754 floating-point numbers are stored as 4 bytes:

```
Bit:  31 | 30-23 | 22-0
      S  |  Exp  | Mantissa
```

When reading a float from N64 RAM:
1. Read 4 bytes in order
2. Interpret as big-endian 32-bit float
3. Convert to Python float

In [ ]:
# Demonstration of endianness

import struct

value = 0x12345678

# Pack as big-endian (N64 format)
big_endian = struct.pack('>I', value)  # '>' means big-endian, 'I' means unsigned int
print(f"Value: 0x{value:08X}")
print(f"\nBig-endian bytes: {' '.join(f'0x{b:02X}' for b in big_endian)}")

# Pack as little-endian (x86 format)
little_endian = struct.pack('<I', value)  # '<' means little-endian
print(f"Little-endian bytes: {' '.join(f'0x{b:02X}' for b in little_endian)}")

# Reading back
print(f"\nReading big-endian bytes as big-endian: 0x{struct.unpack('>I', big_endian)[0]:08X}")
print(f"Reading big-endian bytes as little-endian: 0x{struct.unpack('<I', big_endian)[0]:08X} (wrong!)")

---
## 6. Python struct Module Tutorial

The `struct` module is essential for reading binary data from N64 RAM.

In [ ]:
import struct

print("Python struct Module Tutorial")
print("=" * 50)
print("\nFormat Characters:")
print("  > : big-endian byte order (N64 uses this)")
print("  < : little-endian byte order")
print("  B : unsigned byte (1 byte, 0-255)")
print("  b : signed byte (1 byte, -128 to 127)")
print("  H : unsigned short (2 bytes, 0-65535)")
print("  h : signed short (2 bytes)")
print("  I : unsigned int (4 bytes)")
print("  i : signed int (4 bytes)")
print("  f : float (4 bytes)")

In [ ]:
# Example: Reading different data types

# Simulate N64 RAM bytes (big-endian)
simulated_ram = bytes([
    0x42, 0xC8, 0x00, 0x00,  # Float: 100.0
    0x00, 0x02,              # Unsigned short: 2
    0x03,                    # Unsigned byte: 3
    0x00, 0x00, 0x00, 0x64   # Unsigned int: 100
])

print("Simulated N64 RAM bytes:")
print(f"  {' '.join(f'{b:02X}' for b in simulated_ram)}")

# Read a big-endian float (4 bytes starting at offset 0)
speed = struct.unpack('>f', simulated_ram[0:4])[0]
print(f"\n>f (big-endian float) at offset 0: {speed}")

# Read a big-endian unsigned short (2 bytes starting at offset 4)
lap = struct.unpack('>H', simulated_ram[4:6])[0]
print(f">H (big-endian unsigned short) at offset 4: {lap}")

# Read an unsigned byte (1 byte at offset 6)
position = struct.unpack('>B', simulated_ram[6:7])[0]
print(f">B (unsigned byte) at offset 6: {position}")

# Read a big-endian unsigned int (4 bytes starting at offset 7)
checkpoint = struct.unpack('>I', simulated_ram[7:11])[0]
print(f">I (big-endian unsigned int) at offset 7: {checkpoint}")

In [ ]:
# Creating bytes from values (for testing)

print("Creating bytes from values:")
print("=" * 40)

# Create big-endian float bytes for speed = 45.5
speed_bytes = struct.pack('>f', 45.5)
print(f"\nSpeed 45.5 as big-endian float:")
print(f"  Bytes: {' '.join(f'{b:02X}' for b in speed_bytes)}")
print(f"  Verify: {struct.unpack('>f', speed_bytes)[0]}")

# Create position coordinates
x, y, z = 1250.5, 100.0, -500.25
pos_bytes = struct.pack('>fff', x, y, z)
print(f"\nPosition ({x}, {y}, {z}) as big-endian floats:")
print(f"  Bytes: {' '.join(f'{b:02X}' for b in pos_bytes)}")
print(f"  Verify: {struct.unpack('>fff', pos_bytes)}")

# Create lap counter (byte)
lap_bytes = struct.pack('>B', 2)  # Lap 3 (0-indexed)
print(f"\nLap 2 (0-indexed) as unsigned byte:")
print(f"  Bytes: {' '.join(f'{b:02X}' for b in lap_bytes)}")

---
## 7. MarioKartRAMReader Class Implementation

In [ ]:
import struct
from typing import Optional, Dict, Any, Tuple
from dataclasses import dataclass

@dataclass
class PlayerState:
    """Data class representing player state from RAM."""
    speed: float
    max_speed: float
    position: Tuple[float, float, float]  # x, y, z
    rotation: Tuple[float, float, float]  # x, y, z
    lap: int
    race_position: int
    checkpoint: int
    race_finished: bool
    item: int
    coins: int

class MarioKartRAMReader:
    """
    Reads game state from Mario Kart 64 RAM.
    
    This class provides methods to read various game state values
    from N64 emulator memory. Addresses are for the US NTSC version.
    
    Usage:
        reader = MarioKartRAMReader(memory_interface)
        state = reader.read_player_state(player=1)
        print(f"Speed: {state.speed}")
    """
    
    # N64 virtual address base
    KSEG0_BASE = 0x80000000
    
    # Player 1 addresses (US NTSC)
    ADDR_P1_SPEED = 0x800F6BBC
    ADDR_P1_POS_X = 0x800F6898
    ADDR_P1_POS_Y = 0x800F689C
    ADDR_P1_POS_Z = 0x800F68A0
    ADDR_P1_ROT_X = 0x800F68B0
    ADDR_P1_ROT_Y = 0x800F68B4
    ADDR_P1_ROT_Z = 0x800F68B8
    ADDR_P1_MAX_SPEED = 0x800F6BC0
    ADDR_P1_ITEM = 0x800F68DC
    
    ADDR_CURRENT_LAP = 0x800DC5A0
    ADDR_RACE_POSITION = 0x800DC5AC
    ADDR_CHECKPOINT = 0x800F6860
    ADDR_RACE_FINISHED = 0x800DC598
    ADDR_COINS = 0x800DCDB0
    
    # Player offset for multiplayer
    PLAYER_OFFSET = 0x100
    
    def __init__(self, memory_interface):
        """
        Initialize the RAM reader.
        
        Args:
            memory_interface: Object with read(offset, size) method
                             that reads bytes from emulator RAM.
        """
        self.memory = memory_interface
    
    def _to_offset(self, virtual_addr: int) -> int:
        """Convert N64 virtual address to physical RAM offset."""
        return virtual_addr - self.KSEG0_BASE
    
    def _read_float(self, virtual_addr: int) -> float:
        """Read a big-endian float from the given address."""
        offset = self._to_offset(virtual_addr)
        data = self.memory.read(offset, 4)
        return struct.unpack('>f', data)[0]
    
    def _read_byte(self, virtual_addr: int) -> int:
        """Read an unsigned byte from the given address."""
        offset = self._to_offset(virtual_addr)
        data = self.memory.read(offset, 1)
        return struct.unpack('>B', data)[0]
    
    def _read_short(self, virtual_addr: int) -> int:
        """Read a big-endian unsigned short from the given address."""
        offset = self._to_offset(virtual_addr)
        data = self.memory.read(offset, 2)
        return struct.unpack('>H', data)[0]
    
    def _read_int(self, virtual_addr: int) -> int:
        """Read a big-endian unsigned int from the given address."""
        offset = self._to_offset(virtual_addr)
        data = self.memory.read(offset, 4)
        return struct.unpack('>I', data)[0]
    
    def _player_addr(self, base_addr: int, player: int) -> int:
        """Get address for specific player (1-4)."""
        return base_addr + (player - 1) * self.PLAYER_OFFSET
    
    def read_speed(self, player: int = 1) -> float:
        """Read player's current speed."""
        addr = self._player_addr(self.ADDR_P1_SPEED, player)
        return self._read_float(addr)
    
    def read_max_speed(self, player: int = 1) -> float:
        """Read player's maximum speed."""
        addr = self._player_addr(self.ADDR_P1_MAX_SPEED, player)
        return self._read_float(addr)
    
    def read_position(self, player: int = 1) -> Tuple[float, float, float]:
        """Read player's 3D position (x, y, z)."""
        x = self._read_float(self._player_addr(self.ADDR_P1_POS_X, player))
        y = self._read_float(self._player_addr(self.ADDR_P1_POS_Y, player))
        z = self._read_float(self._player_addr(self.ADDR_P1_POS_Z, player))
        return (x, y, z)
    
    def read_rotation(self, player: int = 1) -> Tuple[float, float, float]:
        """Read player's rotation (x, y, z)."""
        x = self._read_float(self._player_addr(self.ADDR_P1_ROT_X, player))
        y = self._read_float(self._player_addr(self.ADDR_P1_ROT_Y, player))
        z = self._read_float(self._player_addr(self.ADDR_P1_ROT_Z, player))
        return (x, y, z)
    
    def read_lap(self) -> int:
        """Read current lap (0-indexed, so lap 1 = 0)."""
        return self._read_byte(self.ADDR_CURRENT_LAP)
    
    def read_race_position(self) -> int:
        """Read position in race (1-8)."""
        return self._read_byte(self.ADDR_RACE_POSITION)
    
    def read_checkpoint(self, player: int = 1) -> int:
        """Read current checkpoint ID."""
        addr = self._player_addr(self.ADDR_CHECKPOINT, player)
        return self._read_short(addr)
    
    def read_race_finished(self) -> bool:
        """Check if race is finished."""
        return self._read_byte(self.ADDR_RACE_FINISHED) != 0
    
    def read_item(self, player: int = 1) -> int:
        """Read current item held."""
        addr = self._player_addr(self.ADDR_P1_ITEM, player)
        return self._read_byte(addr)
    
    def read_coins(self) -> int:
        """Read coin count."""
        return self._read_byte(self.ADDR_COINS)
    
    def read_player_state(self, player: int = 1) -> PlayerState:
        """Read complete player state."""
        return PlayerState(
            speed=self.read_speed(player),
            max_speed=self.read_max_speed(player),
            position=self.read_position(player),
            rotation=self.read_rotation(player),
            lap=self.read_lap(),
            race_position=self.read_race_position(),
            checkpoint=self.read_checkpoint(player),
            race_finished=self.read_race_finished(),
            item=self.read_item(player),
            coins=self.read_coins()
        )
    
    def get_observation_vector(self, player: int = 1) -> np.ndarray:
        """
        Get state as a numpy array for RL agent input.
        
        Returns:
            Array of shape (13,) containing:
            [speed, max_speed, pos_x, pos_y, pos_z, 
             rot_x, rot_y, rot_z, lap, position, 
             checkpoint, item, coins]
        """
        state = self.read_player_state(player)
        return np.array([
            state.speed,
            state.max_speed,
            state.position[0],
            state.position[1],
            state.position[2],
            state.rotation[0],
            state.rotation[1],
            state.rotation[2],
            state.lap,
            state.race_position,
            state.checkpoint,
            state.item,
            state.coins
        ], dtype=np.float32)

print("MarioKartRAMReader class defined successfully!")

---
## 8. MockMemory Class for Testing

In [ ]:
class MockMemory:
    """
    Mock memory interface for testing RAM reader without emulator.
    
    This class simulates N64 RAM with configurable values for testing.
    """
    
    KSEG0_BASE = 0x80000000
    
    def __init__(self, size: int = 4 * 1024 * 1024):  # 4 MB default
        """
        Initialize mock memory.
        
        Args:
            size: Size of simulated RAM in bytes.
        """
        self.data = bytearray(size)
        self.size = size
    
    def read(self, offset: int, size: int) -> bytes:
        """
        Read bytes from memory.
        
        Args:
            offset: Physical RAM offset
            size: Number of bytes to read
            
        Returns:
            Bytes read from memory
        """
        if offset < 0 or offset + size > self.size:
            raise ValueError(f"Memory access out of bounds: {offset}:{offset+size}")
        return bytes(self.data[offset:offset + size])
    
    def write(self, offset: int, data: bytes):
        """
        Write bytes to memory.
        
        Args:
            offset: Physical RAM offset
            data: Bytes to write
        """
        if offset < 0 or offset + len(data) > self.size:
            raise ValueError(f"Memory access out of bounds: {offset}:{offset+len(data)}")
        self.data[offset:offset + len(data)] = data
    
    def write_float(self, virtual_addr: int, value: float):
        """Write a big-endian float at the given virtual address."""
        offset = virtual_addr - self.KSEG0_BASE
        self.write(offset, struct.pack('>f', value))
    
    def write_byte(self, virtual_addr: int, value: int):
        """Write an unsigned byte at the given virtual address."""
        offset = virtual_addr - self.KSEG0_BASE
        self.write(offset, struct.pack('>B', value))
    
    def write_short(self, virtual_addr: int, value: int):
        """Write a big-endian unsigned short at the given virtual address."""
        offset = virtual_addr - self.KSEG0_BASE
        self.write(offset, struct.pack('>H', value))
    
    def set_player_state(self, speed: float, position: Tuple[float, float, float],
                         rotation: Tuple[float, float, float], lap: int,
                         race_position: int, checkpoint: int, max_speed: float = 50.0,
                         item: int = 0, coins: int = 0, race_finished: bool = False):
        """
        Set complete player state in mock memory.
        
        Convenience method for testing.
        """
        # Speed
        self.write_float(0x800F6BBC, speed)
        self.write_float(0x800F6BC0, max_speed)
        
        # Position
        self.write_float(0x800F6898, position[0])
        self.write_float(0x800F689C, position[1])
        self.write_float(0x800F68A0, position[2])
        
        # Rotation
        self.write_float(0x800F68B0, rotation[0])
        self.write_float(0x800F68B4, rotation[1])
        self.write_float(0x800F68B8, rotation[2])
        
        # Race state
        self.write_byte(0x800DC5A0, lap)
        self.write_byte(0x800DC5AC, race_position)
        self.write_short(0x800F6860, checkpoint)
        self.write_byte(0x800DC598, 1 if race_finished else 0)
        self.write_byte(0x800F68DC, item)
        self.write_byte(0x800DCDB0, coins)

print("MockMemory class defined successfully!")

---
## 9. Test RAM Reader with Mock Data

In [ ]:
# Create mock memory and set initial state
mock_memory = MockMemory()

# Set up a game state
mock_memory.set_player_state(
    speed=45.7,
    position=(1250.5, 100.0, -500.25),
    rotation=(0.0, 0.785, 0.0),  # Facing ~45 degrees
    lap=1,  # Lap 2 (0-indexed)
    race_position=3,  # 3rd place
    checkpoint=42,
    max_speed=50.0,
    item=5,  # Some item
    coins=7
)

# Create RAM reader
reader = MarioKartRAMReader(mock_memory)

# Read and display state
print("Reading game state from mock memory:")
print("=" * 50)

state = reader.read_player_state()
print(f"\nSpeed: {state.speed:.2f} / {state.max_speed:.2f} (max)")
print(f"Position: ({state.position[0]:.2f}, {state.position[1]:.2f}, {state.position[2]:.2f})")
print(f"Rotation: ({state.rotation[0]:.3f}, {state.rotation[1]:.3f}, {state.rotation[2]:.3f})")
print(f"Lap: {state.lap + 1} (raw: {state.lap})")
print(f"Race Position: {state.race_position}")
print(f"Checkpoint: {state.checkpoint}")
print(f"Item: {state.item}")
print(f"Coins: {state.coins}")
print(f"Race Finished: {state.race_finished}")

In [ ]:
# Test observation vector
obs = reader.get_observation_vector()

print("Observation vector for RL:")
print("=" * 50)
print(f"Shape: {obs.shape}")
print(f"Dtype: {obs.dtype}")
print("\nValues:")
labels = ['speed', 'max_speed', 'pos_x', 'pos_y', 'pos_z', 
          'rot_x', 'rot_y', 'rot_z', 'lap', 'position', 
          'checkpoint', 'item', 'coins']
for label, value in zip(labels, obs):
    print(f"  {label:12s}: {value:10.3f}")

---
## 10. Simulated Race Trajectory Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_race_trajectory(n_steps: int = 500) -> Dict[str, np.ndarray]:
    """
    Simulate a race trajectory for visualization.
    
    This creates realistic-looking data for a 3-lap race.
    """
    # Initialize arrays
    speeds = np.zeros(n_steps)
    checkpoints = np.zeros(n_steps, dtype=int)
    laps = np.zeros(n_steps, dtype=int)
    positions_x = np.zeros(n_steps)
    positions_z = np.zeros(n_steps)
    race_positions = np.zeros(n_steps, dtype=int)
    
    # Track parameters
    max_checkpoint = 50  # Checkpoints per lap
    max_speed = 50.0
    track_radius = 1000.0
    
    # Simulate race
    current_checkpoint = 0
    current_lap = 0
    angle = 0
    race_pos = 4  # Start in 4th
    
    for i in range(n_steps):
        # Speed varies during race (acceleration, turns, items)
        base_speed = max_speed * 0.8
        turn_penalty = np.sin(i * 0.1) * 5  # Slow down in turns
        random_variation = np.random.normal(0, 2)
        item_boost = 10 if np.random.random() < 0.02 else 0  # Occasional boost
        speeds[i] = np.clip(base_speed + turn_penalty + random_variation + item_boost, 0, max_speed)
        
        # Progress through checkpoints
        checkpoint_progress = int(i * max_checkpoint * 3 / n_steps) % max_checkpoint
        checkpoints[i] = checkpoint_progress
        
        # Track laps
        laps[i] = min(i * 3 // n_steps, 2)  # 0, 1, or 2
        
        # Circular track position
        angle = 2 * np.pi * (i / (n_steps / 3))  # 3 laps
        positions_x[i] = track_radius * np.cos(angle) + np.random.normal(0, 20)
        positions_z[i] = track_radius * np.sin(angle) + np.random.normal(0, 20)
        
        # Race position changes over time (improve from 4th to 1st)
        progress = i / n_steps
        if progress > 0.8:
            race_pos = 1
        elif progress > 0.5:
            race_pos = 2
        elif progress > 0.3:
            race_pos = 3
        race_positions[i] = race_pos
    
    return {
        'speeds': speeds,
        'checkpoints': checkpoints,
        'laps': laps,
        'positions_x': positions_x,
        'positions_z': positions_z,
        'race_positions': race_positions,
        'time': np.arange(n_steps)
    }

# Generate simulated data
trajectory = simulate_race_trajectory(500)
print("Simulated race trajectory generated!")
print(f"  Steps: {len(trajectory['speeds'])}")
print(f"  Final lap: {trajectory['laps'][-1] + 1}")
print(f"  Final position: {trajectory['race_positions'][-1]}")

In [ ]:
# Visualize the race trajectory

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Speed over time
ax1 = axes[0, 0]
ax1.plot(trajectory['time'], trajectory['speeds'], 'b-', linewidth=0.5, alpha=0.7)
ax1.axhline(y=50, color='r', linestyle='--', label='Max Speed')
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Speed')
ax1.set_title('Speed Over Time')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add lap markers
for lap in range(3):
    lap_start = np.where(trajectory['laps'] == lap)[0]
    if len(lap_start) > 0:
        ax1.axvline(x=lap_start[0], color='g', linestyle=':', alpha=0.5)
        ax1.text(lap_start[0], 52, f'Lap {lap+1}', fontsize=8)

# Plot 2: Checkpoint progress
ax2 = axes[0, 1]
colors = ['blue', 'green', 'red']
for lap in range(3):
    mask = trajectory['laps'] == lap
    ax2.scatter(trajectory['time'][mask], trajectory['checkpoints'][mask], 
                c=colors[lap], s=2, label=f'Lap {lap+1}', alpha=0.6)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('Checkpoint ID')
ax2.set_title('Checkpoint Progress')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Track position (bird's eye view)
ax3 = axes[1, 0]
scatter = ax3.scatter(trajectory['positions_x'], trajectory['positions_z'], 
                      c=trajectory['time'], cmap='viridis', s=2, alpha=0.7)
ax3.set_xlabel('X Position')
ax3.set_ylabel('Z Position')
ax3.set_title('Track Position (Bird\'s Eye View)')
ax3.set_aspect('equal')
plt.colorbar(scatter, ax=ax3, label='Time Step')

# Mark start/finish
ax3.plot(trajectory['positions_x'][0], trajectory['positions_z'][0], 'go', markersize=10, label='Start')
ax3.plot(trajectory['positions_x'][-1], trajectory['positions_z'][-1], 'r*', markersize=15, label='Finish')
ax3.legend()

# Plot 4: Race position over time
ax4 = axes[1, 1]
ax4.plot(trajectory['time'], trajectory['race_positions'], 'r-', linewidth=2)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('Race Position')
ax4.set_title('Race Position Over Time')
ax4.set_ylim(0.5, 8.5)
ax4.set_yticks(range(1, 9))
ax4.invert_yaxis()  # 1st place at top
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('race_trajectory_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nVisualization saved to: race_trajectory_visualization.png")

In [ ]:
# Speed distribution analysis

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histogram of speeds
ax1 = axes[0]
ax1.hist(trajectory['speeds'], bins=30, edgecolor='black', alpha=0.7)
ax1.axvline(x=np.mean(trajectory['speeds']), color='r', linestyle='--', 
            label=f"Mean: {np.mean(trajectory['speeds']):.1f}")
ax1.set_xlabel('Speed')
ax1.set_ylabel('Frequency')
ax1.set_title('Speed Distribution')
ax1.legend()

# Speed by lap
ax2 = axes[1]
lap_speeds = [trajectory['speeds'][trajectory['laps'] == lap] for lap in range(3)]
bp = ax2.boxplot(lap_speeds, labels=['Lap 1', 'Lap 2', 'Lap 3'])
ax2.set_ylabel('Speed')
ax2.set_title('Speed Distribution by Lap')

plt.tight_layout()
plt.show()

print("\nSpeed Statistics:")
print(f"  Mean speed: {np.mean(trajectory['speeds']):.2f}")
print(f"  Max speed: {np.max(trajectory['speeds']):.2f}")
print(f"  Min speed: {np.min(trajectory['speeds']):.2f}")
print(f"  Std deviation: {np.std(trajectory['speeds']):.2f}")

---
## 11. Address Verification Tips

### How to Find RAM Addresses

If you need to find new addresses or verify existing ones:

#### Method 1: Use a Memory Scanner
1. Run the game in an emulator with memory viewing capability
2. Look for values that change predictably (e.g., speed when accelerating)
3. Use "scan for changed/unchanged values" to narrow down

#### Method 2: Check Existing Documentation
- GameHacking.org database
- Speedrunning community wikis
- Tool-assisted speedrun (TAS) resources

#### Method 3: Disassemble the ROM
- Use tools like Ghidra or IDA Pro
- Look for game state structures
- Trace code that updates player state

### Verification Steps

1. **Test known values**: If you know your speed is 0 when stopped, verify the address reads 0
2. **Test boundary conditions**: Max speed, lap transitions, etc.
3. **Cross-reference**: Compare with multiple sources
4. **Version check**: Verify ROM version matches address documentation

### Common Pitfalls

- **Wrong ROM version**: US, PAL, and Japanese versions have different addresses
- **Endianness errors**: N64 is big-endian, x86 is little-endian
- **Address space confusion**: Virtual vs physical addresses
- **Data type mismatch**: Reading a float as int or vice versa

In [ ]:
# Address verification helper

def verify_address(reader, address_name: str, expected_range: tuple, 
                   actual_value: float) -> bool:
    """
    Verify an address reads a value in the expected range.
    
    Args:
        reader: MarioKartRAMReader instance
        address_name: Name of the value being checked
        expected_range: (min, max) tuple of expected values
        actual_value: The value read from memory
        
    Returns:
        True if value is in expected range
    """
    min_val, max_val = expected_range
    is_valid = min_val <= actual_value <= max_val
    
    status = "PASS" if is_valid else "FAIL"
    print(f"[{status}] {address_name}: {actual_value:.2f} "
          f"(expected: {min_val:.2f} - {max_val:.2f})")
    
    return is_valid

# Run verification checks
print("Address Verification Report")
print("=" * 50)

state = reader.read_player_state()

all_passed = all([
    verify_address(reader, "Speed", (0, 100), state.speed),
    verify_address(reader, "Max Speed", (30, 80), state.max_speed),
    verify_address(reader, "Lap", (0, 2), state.lap),
    verify_address(reader, "Race Position", (1, 8), state.race_position),
    verify_address(reader, "Checkpoint", (0, 100), state.checkpoint),
])

print("\n" + "=" * 50)
if all_passed:
    print("All verification checks PASSED!")
else:
    print("Some verification checks FAILED. Review addresses.")

---
## 12. Knowledge Check Quiz

In [ ]:
def ram_reading_quiz():
    """
    Interactive quiz to test understanding of RAM reading.
    """
    questions = [
        {
            "question": "What is the main advantage of RAM reading over screen pixels for RL?",
            "options": [
                "A) Works for any game",
                "B) Lower dimensional input with precise values",
                "C) No reverse engineering needed",
                "D) Better visual appearance"
            ],
            "correct": "B",
            "explanation": "RAM reading provides exact numerical values in a low-dimensional vector, eliminating the need for CNN processing and giving precise state information."
        },
        {
            "question": "What byte order does the N64 use?",
            "options": [
                "A) Little-endian",
                "B) Big-endian",
                "C) Mixed-endian",
                "D) Network order"
            ],
            "correct": "B",
            "explanation": "The N64's MIPS CPU uses big-endian byte order, where the most significant byte is stored first."
        },
        {
            "question": "What does the struct format '>f' mean in Python?",
            "options": [
                "A) Little-endian float",
                "B) Big-endian float",
                "C) Forward float",
                "D) File float"
            ],
            "correct": "B",
            "explanation": "The '>' indicates big-endian byte order, and 'f' indicates a 4-byte float."
        },
        {
            "question": "Why do N64 game addresses often start with 0x80?",
            "options": [
                "A) The N64 only has 128 bytes of RAM",
                "B) It's a magic number for game detection",
                "C) KSEG0 virtual memory space maps to physical RAM",
                "D) 0x80 is the checksum prefix"
            ],
            "correct": "C",
            "explanation": "KSEG0 (0x80000000-0x9FFFFFFF) is a cached virtual memory segment that maps directly to physical RAM."
        },
        {
            "question": "What should you do if RAM addresses don't work?",
            "options": [
                "A) Use random addresses instead",
                "B) Verify ROM version matches documented addresses",
                "C) Switch to little-endian",
                "D) Use larger data types"
            ],
            "correct": "B",
            "explanation": "Different ROM versions (US, PAL, JP) have different memory layouts. Always verify your ROM version matches the address documentation."
        }
    ]
    
    score = 0
    total = len(questions)
    
    print("=" * 60)
    print("RAM READING QUIZ")
    print("=" * 60)
    print("\nAnswer each question by entering A, B, C, or D.\n")
    
    for i, q in enumerate(questions, 1):
        print(f"\nQuestion {i}/{total}: {q['question']}")
        for option in q['options']:
            print(f"  {option}")
        
        while True:
            try:
                answer = input("\nYour answer: ").strip().upper()
                if answer in ['A', 'B', 'C', 'D']:
                    break
                print("Please enter A, B, C, or D")
            except:
                print("Input error. Using placeholder answer.")
                answer = q['correct']
                break
        
        if answer == q['correct']:
            print("Correct!")
            score += 1
        else:
            print(f"Incorrect. The correct answer is {q['correct']}.")
        
        print(f"Explanation: {q['explanation']}")
    
    print("\n" + "=" * 60)
    print(f"QUIZ COMPLETE!")
    print(f"Your score: {score}/{total} ({100*score/total:.0f}%)")
    print("=" * 60)
    
    if score == total:
        print("\nPerfect score! You have a solid understanding of RAM reading!")
    elif score >= total * 0.7:
        print("\nGood job! Review the topics you missed.")
    else:
        print("\nConsider reviewing this notebook again.")

# Run the quiz
print("To take the quiz, uncomment and run: ram_reading_quiz()")
# ram_reading_quiz()

---
## 13. Save RAM Reader Module

In [ ]:
# Save the RAM reader as a reusable module

ram_reader_code = '''
"""
Mario Kart 64 RAM Reader Module

This module provides classes for reading game state from N64 emulator memory.
Designed for use with Mupen64Plus and gym-mupen64plus.

Usage:
    from ram_reader import MarioKartRAMReader, MockMemory
    
    # For testing
    memory = MockMemory()
    reader = MarioKartRAMReader(memory)
    
    # With real emulator (pseudo-code)
    # memory = EmulatorMemoryInterface()
    # reader = MarioKartRAMReader(memory)
    # state = reader.read_player_state()
"""

import struct
import numpy as np
from typing import Tuple, Dict, Any
from dataclasses import dataclass


@dataclass
class PlayerState:
    """Data class representing player state from RAM."""
    speed: float
    max_speed: float
    position: Tuple[float, float, float]  # x, y, z
    rotation: Tuple[float, float, float]  # x, y, z
    lap: int
    race_position: int
    checkpoint: int
    race_finished: bool
    item: int
    coins: int


class MarioKartRAMReader:
    """
    Reads game state from Mario Kart 64 RAM.
    
    This class provides methods to read various game state values
    from N64 emulator memory. Addresses are for the US NTSC version.
    """
    
    # N64 virtual address base
    KSEG0_BASE = 0x80000000
    
    # Player 1 addresses (US NTSC)
    ADDR_P1_SPEED = 0x800F6BBC
    ADDR_P1_POS_X = 0x800F6898
    ADDR_P1_POS_Y = 0x800F689C
    ADDR_P1_POS_Z = 0x800F68A0
    ADDR_P1_ROT_X = 0x800F68B0
    ADDR_P1_ROT_Y = 0x800F68B4
    ADDR_P1_ROT_Z = 0x800F68B8
    ADDR_P1_MAX_SPEED = 0x800F6BC0
    ADDR_P1_ITEM = 0x800F68DC
    
    ADDR_CURRENT_LAP = 0x800DC5A0
    ADDR_RACE_POSITION = 0x800DC5AC
    ADDR_CHECKPOINT = 0x800F6860
    ADDR_RACE_FINISHED = 0x800DC598
    ADDR_COINS = 0x800DCDB0
    
    PLAYER_OFFSET = 0x100
    
    def __init__(self, memory_interface):
        """Initialize with a memory interface that has read(offset, size) method."""
        self.memory = memory_interface
    
    def _to_offset(self, virtual_addr: int) -> int:
        """Convert N64 virtual address to physical RAM offset."""
        return virtual_addr - self.KSEG0_BASE
    
    def _read_float(self, virtual_addr: int) -> float:
        """Read a big-endian float from the given address."""
        offset = self._to_offset(virtual_addr)
        data = self.memory.read(offset, 4)
        return struct.unpack(\'>f\', data)[0]
    
    def _read_byte(self, virtual_addr: int) -> int:
        """Read an unsigned byte from the given address."""
        offset = self._to_offset(virtual_addr)
        data = self.memory.read(offset, 1)
        return struct.unpack(\'>B\', data)[0]
    
    def _read_short(self, virtual_addr: int) -> int:
        """Read a big-endian unsigned short from the given address."""
        offset = self._to_offset(virtual_addr)
        data = self.memory.read(offset, 2)
        return struct.unpack(\'>H\', data)[0]
    
    def _player_addr(self, base_addr: int, player: int) -> int:
        """Get address for specific player (1-4)."""
        return base_addr + (player - 1) * self.PLAYER_OFFSET
    
    def read_speed(self, player: int = 1) -> float:
        """Read player\'s current speed."""
        addr = self._player_addr(self.ADDR_P1_SPEED, player)
        return self._read_float(addr)
    
    def read_max_speed(self, player: int = 1) -> float:
        """Read player\'s maximum speed."""
        addr = self._player_addr(self.ADDR_P1_MAX_SPEED, player)
        return self._read_float(addr)
    
    def read_position(self, player: int = 1) -> Tuple[float, float, float]:
        """Read player\'s 3D position (x, y, z)."""
        x = self._read_float(self._player_addr(self.ADDR_P1_POS_X, player))
        y = self._read_float(self._player_addr(self.ADDR_P1_POS_Y, player))
        z = self._read_float(self._player_addr(self.ADDR_P1_POS_Z, player))
        return (x, y, z)
    
    def read_rotation(self, player: int = 1) -> Tuple[float, float, float]:
        """Read player\'s rotation (x, y, z)."""
        x = self._read_float(self._player_addr(self.ADDR_P1_ROT_X, player))
        y = self._read_float(self._player_addr(self.ADDR_P1_ROT_Y, player))
        z = self._read_float(self._player_addr(self.ADDR_P1_ROT_Z, player))
        return (x, y, z)
    
    def read_lap(self) -> int:
        """Read current lap (0-indexed)."""
        return self._read_byte(self.ADDR_CURRENT_LAP)
    
    def read_race_position(self) -> int:
        """Read position in race (1-8)."""
        return self._read_byte(self.ADDR_RACE_POSITION)
    
    def read_checkpoint(self, player: int = 1) -> int:
        """Read current checkpoint ID."""
        addr = self._player_addr(self.ADDR_CHECKPOINT, player)
        return self._read_short(addr)
    
    def read_race_finished(self) -> bool:
        """Check if race is finished."""
        return self._read_byte(self.ADDR_RACE_FINISHED) != 0
    
    def read_item(self, player: int = 1) -> int:
        """Read current item held."""
        addr = self._player_addr(self.ADDR_P1_ITEM, player)
        return self._read_byte(addr)
    
    def read_coins(self) -> int:
        """Read coin count."""
        return self._read_byte(self.ADDR_COINS)
    
    def read_player_state(self, player: int = 1) -> PlayerState:
        """Read complete player state."""
        return PlayerState(
            speed=self.read_speed(player),
            max_speed=self.read_max_speed(player),
            position=self.read_position(player),
            rotation=self.read_rotation(player),
            lap=self.read_lap(),
            race_position=self.read_race_position(),
            checkpoint=self.read_checkpoint(player),
            race_finished=self.read_race_finished(),
            item=self.read_item(player),
            coins=self.read_coins()
        )
    
    def get_observation_vector(self, player: int = 1) -> np.ndarray:
        """
        Get state as a numpy array for RL agent input.
        
        Returns:
            Array of shape (13,) containing normalized state values.
        """
        state = self.read_player_state(player)
        return np.array([
            state.speed,
            state.max_speed,
            state.position[0],
            state.position[1],
            state.position[2],
            state.rotation[0],
            state.rotation[1],
            state.rotation[2],
            state.lap,
            state.race_position,
            state.checkpoint,
            state.item,
            state.coins
        ], dtype=np.float32)


class MockMemory:
    """
    Mock memory interface for testing RAM reader without emulator.
    """
    
    KSEG0_BASE = 0x80000000
    
    def __init__(self, size: int = 4 * 1024 * 1024):
        """Initialize mock memory with given size in bytes."""
        self.data = bytearray(size)
        self.size = size
    
    def read(self, offset: int, size: int) -> bytes:
        """Read bytes from memory."""
        if offset < 0 or offset + size > self.size:
            raise ValueError(f"Memory access out of bounds: {offset}:{offset+size}")
        return bytes(self.data[offset:offset + size])
    
    def write(self, offset: int, data: bytes):
        """Write bytes to memory."""
        if offset < 0 or offset + len(data) > self.size:
            raise ValueError(f"Memory access out of bounds")
        self.data[offset:offset + len(data)] = data
    
    def write_float(self, virtual_addr: int, value: float):
        """Write a big-endian float at the given virtual address."""
        offset = virtual_addr - self.KSEG0_BASE
        self.write(offset, struct.pack(\'>f\', value))
    
    def write_byte(self, virtual_addr: int, value: int):
        """Write an unsigned byte at the given virtual address."""
        offset = virtual_addr - self.KSEG0_BASE
        self.write(offset, struct.pack(\'>B\', value))
    
    def write_short(self, virtual_addr: int, value: int):
        """Write a big-endian unsigned short at the given virtual address."""
        offset = virtual_addr - self.KSEG0_BASE
        self.write(offset, struct.pack(\'>H\', value))
    
    def set_player_state(self, speed: float, position: Tuple[float, float, float],
                         rotation: Tuple[float, float, float], lap: int,
                         race_position: int, checkpoint: int, max_speed: float = 50.0,
                         item: int = 0, coins: int = 0, race_finished: bool = False):
        """Set complete player state in mock memory for testing."""
        self.write_float(0x800F6BBC, speed)
        self.write_float(0x800F6BC0, max_speed)
        self.write_float(0x800F6898, position[0])
        self.write_float(0x800F689C, position[1])
        self.write_float(0x800F68A0, position[2])
        self.write_float(0x800F68B0, rotation[0])
        self.write_float(0x800F68B4, rotation[1])
        self.write_float(0x800F68B8, rotation[2])
        self.write_byte(0x800DC5A0, lap)
        self.write_byte(0x800DC5AC, race_position)
        self.write_short(0x800F6860, checkpoint)
        self.write_byte(0x800DC598, 1 if race_finished else 0)
        self.write_byte(0x800F68DC, item)
        self.write_byte(0x800DCDB0, coins)


if __name__ == "__main__":
    # Test the module
    print("Testing RAM Reader Module")
    print("=" * 40)
    
    memory = MockMemory()
    memory.set_player_state(
        speed=45.0,
        position=(1000.0, 100.0, -500.0),
        rotation=(0.0, 0.5, 0.0),
        lap=1,
        race_position=2,
        checkpoint=25
    )
    
    reader = MarioKartRAMReader(memory)
    state = reader.read_player_state()
    
    print(f"Speed: {state.speed}")
    print(f"Position: {state.position}")
    print(f"Lap: {state.lap + 1}")
    print(f"Race Position: {state.race_position}")
    print("\nModule test passed!")
'''

# Save to file
with open('ram_reader.py', 'w') as f:
    f.write(ram_reader_code)

print("RAM reader module saved to: ram_reader.py")
print("\nYou can import it with:")
print("  from ram_reader import MarioKartRAMReader, MockMemory, PlayerState")

In [ ]:
# Verify the saved module works
!python ram_reader.py

---
## Summary

In this notebook, we learned:

1. **RAM Reading Advantages**: Lower dimensional input, exact values, no CNN needed
2. **N64 Memory Layout**: KSEG0 virtual memory, 4MB RDRAM
3. **Mario Kart 64 Addresses**: Speed, position, lap, checkpoint, and more
4. **Big-Endian Byte Order**: N64 stores bytes MSB-first
5. **struct Module**: Reading binary data with >f, >B, >H, >I formats
6. **MarioKartRAMReader**: Complete class for reading all game state
7. **MockMemory**: Testing without emulator
8. **Visualization**: Analyzing race trajectories from RAM data

### Key Takeaways

- RAM reading provides **precise, low-dimensional state** for RL
- Always use **big-endian** format when reading N64 memory
- **Verify addresses** against your specific ROM version
- The `get_observation_vector()` method provides ready-to-use RL input

### Next Steps

1. Connect the RAM reader to the actual emulator
2. Design reward functions using RAM state
3. Build the complete RL environment

---

*The saved `ram_reader.py` module can be imported in future notebooks.*